In [1]:
line = [
    [10.171054833, 75.870382],
    [11.961542, 76.823127]
]
circles = [{"center": [10.171054833622044, 75.87038261100483],
  "radius": 5.251052195950642,
  "strip": [0.5,0.2,0.1],
  "trust": 75},
 {"center": [10.337583, 75.420917],
  "radius":  10.530432089900206,
  "strip": [0.5,0.2,0.1],
  "trust": 57},
	 {"center": [10.420933, 75.872667],
  "radius":   4.686149805543993,
  "strip": [0.5,0.2,0.1],
  "trust": 25}
	 ]
N = 10
width = 100

In [59]:
import numpy as np
from numpy import arctan2,random,sin,cos,degrees
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils import *
from temp import *
import folium


In [60]:
item=  {
        "data": [
            [29.961542, 76.823127],
            [32.961542, 76.823127],
            100
        ],
        "circle": [{
            "center": [29.961542, 76.823127],
            "radius": 15,
            "trust": 69
        },
        {
            "center": [29.961542, 79.823127],
            "radius": 20,
            "trust": 69
        }]
    }

In [61]:
circles = item["circle"]
m = folium.Map(zoom_start=5, location=circles[0]["center"],  tiles="CartoDB dark_matter")
if circles is not None: # draw the circles
    all_points = []
    for circle in circles:
        circle["rad_strips"] = []
        points, points1, points2 = [], [], []

        x, y = circle["center"]

        points.extend(PointsInCircum(x, y, circle['radius']*1*0.02, n=100)) #6,092 km
        circle["rad_strips"].append(cal_dist(*points[0], *circle["center"]))
        points1.extend(PointsInCircum(x, y, circle['radius']*2*0.02, n=100))
        circle["rad_strips"].append(cal_dist(*points1[0], *circle["center"]))
        points2.extend(PointsInCircum(x, y, circle['radius']*3*0.02, n=100))
        circle["rad_strips"].append(cal_dist(*points2[0], *circle["center"]))

        all_points.extend(points)
        all_points.extend(points1)
        all_points.extend(points2)

        folium.PolyLine(points, color="green", popup=points[0]).add_to(m)
        folium.PolyLine(points1, color="red", popup=points1[0]).add_to(m)
        folium.PolyLine(points2, color="blue", popup=points2[0]).add_to(m)

    xx, yy = zip(*all_points)
    min_x = min(xx)
    min_y = min(yy)
    max_x = max(xx)
    max_y = max(yy)
    bbox = [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]
    bbox += [bbox[0]]

    folium.PolyLine(bbox).add_to(m)

In [62]:
m

# Getting the Grid Ready

In [82]:
centers = circles

In [96]:
N = 11
# 1 Line And Circle Datum Grid
lower_left, upper_right = bbox[0], bbox[2]
grid = get_geojson_grid(upper_right, lower_left, n=N)
for _grid in grid:
    _grid["prob_dist"] = helper(_grid, centers)
    color = plt.cm.Reds(_grid["prob_dist"])
    color = mpl.colors.to_hex(color)
    _grid["color"] = color

In [97]:
_grid

{'type': 'FeatureCollection',
 'properties': {'lower_left': [30.943360181818182, 80.55949063636363],
  'upper_right': [31.161542, 81.023127]},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[80.55949063636363, 31.161542],
      [81.023127, 31.161542],
      [81.023127, 30.943360181818182],
      [80.55949063636363, 30.943360181818182],
      [80.55949063636363, 31.161542]]]}}],
 'prob_dist': 0,
 'color': '#fff5f0'}

In [98]:
# m = folium.Map(zoom_start=5, location=[
#                55, 0],  tiles="CartoDB dark_matter")

for i, geo_json in enumerate(grid):
    color = geo_json["color"]

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })
#         popup = folium.Popup(f"{geo_json['prob_dist']}")

#     popup = folium.Tooltip(f"={circle_nos}-{helper(geo_json, centers)}")
    popup = folium.Tooltip(f"{geo_json['prob_dist']}")
    gj.add_child(popup)
    m.add_child(gj)

In [99]:
m

In [34]:
centers, distances

([{'center': [29.961542, 76.823127],
   'radius': 15,
   'trust': 69,
   'rad_strips': [0.3000000000000007, 0.6000000000000014, 0.8999999999999986]},
  {'center': [29.961542, 79.823127],
   'radius': 20,
   'trust': 69,
   'rad_strips': [0.3999999999999986,
    0.8000000000000007,
    1.1999999999999993]}],
 [1.2792769781313433, 3.826962306564802])

In [45]:
distances = [0.2, 3.2]

In [76]:
def helper(geo_json, centers):
    _points = geo_json["features"][0]["geometry"]["coordinates"][0][:-1]
    xx, yy = zip(*_points)
    centroid = (sum(xx) / len(_points), sum(yy) / len(_points))[::-1]
    distances = []
    for center in centers:
        distance = cal_dist(*centroid, *center["center"])
        distances.append(distance)
    circle_nos = distances.index(min(distances))
    min_dist = min(distances)
    circle_nos = distances.index(min_dist)
    radius = centers[circle_nos]["rad_strips"]

    prob = 0
    if min_dist <= radius[0]:
        prob = 0.5
    elif min_dist >= radius[0] and min_dist <= radius[1]:
        prob = 0.3
    elif min_dist >= radius[1] and min_dist <= radius[2]:
        prob = 0.17
    return prob*centers[circle_nos]["trust"]

In [52]:
min(radius) > min_dist

True